<a href="https://colab.research.google.com/github/mchivuku/csb659-project/blob/master/VAE_%2B_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
!pip install tqdm six

In [2]:
from google.colab import drive

drive.mount("/content/drive")


%cd /content/drive/My\ Drive/Masters-DS/CSCI-B659/project/examples/vae/
%ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Masters-DS/CSCI-B659/project/examples/vae
MNIST/  results/


## Classification Accuracy of VAE

In [1]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image


params = {
    "batch_size":128,
    "epochs" : 10,
    "log_interval":10
    
}

torch.manual_seed(5)

print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', torch.device('cuda:0'))
print(torch.cuda.is_available())

is_cuda = torch.cuda.is_available()
device = torch.device ( "cuda:0" if torch.cuda.is_available () else "cpu" )




Torch 1.0.1.post2 CUDA 10.0.130
Device: cuda:0
True


In [0]:
class RunningAverage ():
    """A simple class that maintains the running average of a quantity
    Example:
    ```
    loss_avg = RunningAverage()
    loss_avg.update(2)
    loss_avg.update(4)
    loss_avg() = 3
    ```
    """

    def __init__( self ):
        self.steps = 0
        self.total = 0

    def update( self, val ):
        self.total += val
        self.steps += 1
    
    def reset(self):
        self.steps = 0.
        self.total = 0.
        
        
        
    def __call__( self ):
        return self.total / float ( self.steps )

In [11]:
## Data Loaders
kwargs = {'num_workers': 1, 'pin_memory': True}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('MNIST/data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=params.get("batch_size"), shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('MNIST/data', train=False, transform=transforms.ToTensor()),
    batch_size=params.get("batch_size"), shuffle=True, **kwargs)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 19829563.97it/s]                            


Extracting MNIST/data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 299560.49it/s]                           
0it [00:00, ?it/s]

Extracting MNIST/data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 5204865.37it/s]                           
8192it [00:00, 124379.14it/s]


Extracting MNIST/data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting MNIST/data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [48]:
from torch.nn import init

## Classifier
class Classifier(nn.Module):
  def __init__(self,dims):
    """
    Single hidden layer with softmax
    """
    super(Classifier,self).__init__()
    [x_dim, h_dim, y_dim] = dims
    
    self.fc1 = nn.Linear(x_dim, h_dim)
    self.fc2 = nn.Linear(h_dim, y_dim)
    
  def forward(self,x):
    ## flatten x input
    x = x.view(-1,self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.softmax(self.fc2(x),dim=-1)
    
    return x
  
  
  def num_flat_features(self,x):
    size = x.size()[1:] # all dimensions except the batch dimension
    num_features = 1
    for s in size:
      num_features *=s
    return num_features
  
"""
Gaussian Sample Layer
"""
class GaussianSample(nn.Module):
  def __init__(self,in_features, out_features):
    super(GaussianSample, self).__init__()
    self.in_features = in_features
    self.out_features = out_features
    
    self.mu = nn.Linear(in_features, out_features)
    self.log_var = nn.Linear(in_features, out_features)
    
  def forward(self,x):
    mu = self.mu(x)
    log_var = F.softplus(self.logvar(x))
    
    return self.reparametrize(mu,log_var), mu, log_var
    
  def reparametrize(self, mu, log_var):
    epsilon = Variable(torch.randn(mu.size()), requires_grad=False)

    if mu.is_cuda:
      epsilon = epsilon.cuda()
    # log_std = 0.5 * log_var
    # std = exp(log_std)
    std = log_var.mul(0.5).exp_()

    # z = std * epsilon + mu
    z = mu.addcmul(std, epsilon)
    return z
      

"""
VAE Encoder
"""
class Encoder(nn.Module):
  def __init__(self,dims):
    super(Encoder,self).__init__()
    [x_dim, h_dim, z_dim] = dims
    ##linear layers
    neurons = [x_dim, *h_dim]
    linear_layers = [nn.Linear(neurons[i-1],neurons[i]) for i in range(1,len(neurons))]
    
    self.hidden=nn.ModuleList(linear_layers)
    
    self.sample = GaussianSample(h_dim[-1],z_dim)
    
    
  def forward(self,x):
    for layer in self.hidden:
      x = F.relu(layer(x))
    return self.sample(x)
  
"""
VAE Decoder
"""
class Decoder(nn.Module):
  def __init__(self,dims):
    super(Decoder,self).__init__()
    [z_dim, h_dim, x_dim] = dims
    ##linear layers
    neurons = [z_dim, *h_dim]
    linear_layers = [nn.Linear(neurons[i-1],neurons[i]) for i in range(1,len(neurons))]
    
    self.hidden=nn.ModuleList(linear_layers)
    
    self.reconstruction = nn.Linear(h_dim[-1],x_dim)
    self.output_activation = torch.sigmoid
    
    
  def forward(self,x):
    for layer in self.hidden:
      x = F.relu(layer(x))
    return self.output_activation(self.reconstruction(x))
  


"""
VAE object
"""
class VAE(nn.Module):
  def __init__(self,dims):
    super(VAE,self).__init__()
    
    [x_dim,z_dim,h_dim] = dims
    self.z_dim = z_dim
    self.h_dim = h_dim
    self.x_dim = x_dim
    
    
    ## Encoder
    self.encoder = Encoder([x_dim,h_dim,z_dim])
    ## Decoder
    self.decoder = Decoder([z_dim,h_dim,x_dim])
    
    ## xaview initialization for weights
    for m in self.modules():
      if isinstance(m, nn.Linear):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
          m.bias.data.zero_()
    
 
  """
  Forward
  """
  def forward(self,x):
    z, z_mu, z_log_var = self.encoder(x)
    return self.decoder(z), z_mu, z_log_var
  
  
  
"""
Deep Generative Model 
"""
class DeepGenerativeModel(VAE):
    def __init__(self, dims):
        """
        M2 code replication from the paper
        'Semi-Supervised Learning with Deep Generative Models'
        (Kingma 2014) in PyTorch.
        The "Generative semi-supervised model" is a probabilistic
        model that incorporates label information in both
        inference and generation.
        Initialise a new generative model
        :param dims: dimensions of x, y, z and hidden layers.
        """
        super(DeepGenerativeModel, self).__init__([dims[0], dims[2], dims[3]])
        [x_dim, self.y_dim, z_dim, h_dim] = dims
        

        self.encoder = Encoder([x_dim + self.y_dim, h_dim, z_dim])
        self.decoder = Decoder([z_dim + self.y_dim, list(reversed(h_dim)), x_dim])
        self.classifier = Classifier([x_dim, h_dim[0], self.y_dim])

        for m in self.modules():
            if isinstance(m, nn.Linear):
              init.xavier_normal_(m.weight.data)
              if m.bias is not None:
                m.bias.data.zero_()

    """
    Compute KLD on gaussian sample - inference
    """
    def _kld(self,x,q_param):
      (mu, log_var) = q_param
      log_pdf = - 0.5 * math.log(2 * math.pi) - log_var / 2 - (x - mu)**2 / (2 * torch.exp(log_var))
      return torch.sum(log_pdf, dim=-1)
      
    def forward(self, x, y):
        # Add label and data and generate latent variable
        z, z_mu, z_log_var = self.encoder(torch.cat([x, y], dim=1))

        self.kl_divergence = self._kld(z, (z_mu, z_log_var))

        # Reconstruct data point from latent data and label
        x_mu = self.decoder(torch.cat([z, y], dim=1))

        return x_mu

    def classify(self, x):
        logits = self.classifier(x)
        return logits

    def sample(self, z, y):
        """
        Samples from the Decoder to generate an x.
        :param z: latent normal variable
        :param y: label (one-hot encoded)
        :return: x
        """
        y = y.float()
        x = self.decoder(torch.cat([z, y], dim=1))
        return x

"""
Joint loss over all the parameters
"""
def loss_function(recon_x, x, mu, logvar):
  BCE = F.binary_cross_entropy(recon_x,x.view(-1,784),reduction="sum")
  # https://arxiv.org/abs/1312.6114
  # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
  KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
  
  return BCE + KLD
  
y_dim = 10
z_dim = 32
h_dim = [256, 128]

model = DeepGenerativeModel([784, y_dim, z_dim, h_dim]).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


model
    

DeepGenerativeModel(
  (encoder): Encoder(
    (hidden): ModuleList(
      (0): Linear(in_features=794, out_features=256, bias=True)
      (1): Linear(in_features=256, out_features=128, bias=True)
    )
    (sample): GaussianSample(
      (mu): Linear(in_features=128, out_features=32, bias=True)
      (log_var): Linear(in_features=128, out_features=32, bias=True)
    )
  )
  (decoder): Decoder(
    (hidden): ModuleList(
      (0): Linear(in_features=42, out_features=128, bias=True)
      (1): Linear(in_features=128, out_features=256, bias=True)
    )
    (reconstruction): Linear(in_features=256, out_features=784, bias=True)
  )
  (classifier): Classifier(
    (fc1): Linear(in_features=784, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=10, bias=True)
  )
)

In [0]:
"""
Train
"""
def train(epoch):
  model.train()
  train_loss = 0
  accuracy = RunningAverage()
  for batch_idx, (data, labels) in enumerate(train_loader):
    data = data.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    batch_size = data.size(0)
    recon_batch, mu, logvar = model(data)
    loss = loss_function(recon_batch, data, mu, logvar)
    loss.backward()
    
    train_loss+=loss.item()
    
    
    ## accuracy
    _, idx = recon_batch.topk(1,dim=1)
    accuracy.update(torch.sum(idx.view(batch_size) == labels).item())
    print(idx)    
    
    optimizer.step()
    if batch_idx % params.get("log_interval") == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))
    tl = train_loss/len(train_loader.dataset)
    print("==> Epoch {}, average loss: {:.4f},average accuracy: {:.4f}".format(epoch, tl,accuracy()))
    
    return (tl,accuracy)

  

In [0]:
"""
Test
"""
def test(epoch):
    model.eval()
    test_loss = 0
    test_accuracy = RunningAverage()
    with torch.no_grad():
        for i, (data,labels) in enumerate(test_loader):
            data = data.to(device)
            labels = labels.to(device)
            recon_batch, mu, logvar = model(data)
            batch_size = data.size(0)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            ## accuracy
            _, idx = recon_batch.topk(1,dim=1)
            test_accuracy.update(torch.sum(idx.view(batch_size) == labels).item())
        
            
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(params.get("batch_size"), 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         './results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}, test accuracy: {:.4f}'.format(test_loss,test_accuracy()))
    return (test_loss,test_accuracy)

In [35]:
train_losses = []
test_losses = []
train_acc_list = []
test_acc_list = []
for epoch in range(1, 2):
        trainloss,train_acc = train(epoch)
        testloss,test_acc = test(epoch)
        train_losses.append(trainloss)
        test_losses.append(testloss)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        
        with torch.no_grad():
            sample = torch.randn(64, 20).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, 28, 28),
                       'results/sample_' + str(epoch) + '.png')

tensor([[407],
        [408],
        [408],
        [407],
        [408],
        [434],
        [407],
        [407],
        [408],
        [407],
        [407],
        [407],
        [407],
        [407],
        [408],
        [434],
        [407],
        [407],
        [408],
        [407],
        [408],
        [407],
        [407],
        [407],
        [407],
        [407],
        [408],
        [407],
        [407],
        [408],
        [408],
        [407],
        [408],
        [407],
        [408],
        [434],
        [407],
        [407],
        [408],
        [408],
        [407],
        [407],
        [407],
        [407],
        [407],
        [407],
        [408],
        [407],
        [407],
        [407],
        [407],
        [408],
        [407],
        [408],
        [407],
        [407],
        [408],
        [407],
        [407],
        [407],
        [407],
        [408],
        [407],
        [407],
        [407],
        [407],
        [4

In [0]:
import os
os.makedirs("./results")